In [1]:
import pandas as pd
import numpy as np
import datetime

In [113]:
df = pd.read_csv('Wrangling_data.csv')

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
ts            3553 non-null object
user_id       3553 non-null object
country_id    3553 non-null object
site_id       3553 non-null object
dtypes: object(4)
memory usage: 111.1+ KB


### Site Id with largest # of unique users

In [115]:
df_BDV = df[df.country_id == 'BDV']

In [116]:
df_BDV.shape

(844, 4)

In [117]:
df.head()

,ts,user_id,country_id,site_id
0,2/1/19 0:01,LC36FC,TL6,N0OTG
1,2/1/19 0:10,LC39B6,TL6,N0OTG
2,2/1/19 0:21,LC3500,TL6,N0OTG
3,2/1/19 0:22,LC374F,TL6,N0OTG
4,2/1/19 0:23,LCC1C3,TL6,QGO3G


#### Grouping site ids and counting number of unique users

In [118]:
df_BDV_site = df[df.country_id == 'BDV'].groupby('site_id')['user_id'].nunique().to_frame()
#soring by no of users and get top value
df_BDV_site.sort_values(by = 'user_id', ascending = False).head(1)

,user_id
site_id,
5NPAU,544


### Users between timestamps and the sites they visited more than 10 times

In [127]:
# convert string to timestamp and get all timestamps in between
df['ts'] = pd.to_datetime(df['ts'])
start_time = datetime.datetime.strptime('2019-02-03 00:00:00','%Y-%m-%d %H:%M:%S')
end_time = datetime.datetime.strptime('2019-02-04 23:59:59','%Y-%m-%d %H:%M:%S')
df_time = df[(df.ts > start_time) & (df.ts < end_time)]

In [128]:
#count number of times a user visited a site by counting timestamps
df_time = df_time.groupby(['user_id','site_id'])['ts'].count().to_frame()
df_time[df_time.ts > 10].sort_values(by = 'ts', ascending = False)

,,ts
user_id,site_id,
LC3A59,N0OTG,26
LC06C3,N0OTG,25
LC3C9D,N0OTG,17
LC3C7E,3POLC,15


### Top 3 sites with largest unique numbers whose last visit was to that site

In [121]:
#get last timestamp for each user
df_user = df.groupby('user_id')['ts'].max().reset_index()

In [122]:
#get sites that users visited last, group by site and count unique number of users, get top 3 by sorting
pd.merge(df_user,df, how='inner',on = ['ts','user_id']).groupby('site_id')['user_id'].nunique().to_frame().sort_values('user_id', ascending = False)

,user_id
site_id,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


### Number of users whose first and last visits are to the same site

In [123]:
# get first and last timestamps of each user
df_first = df.groupby('user_id')['ts'].min().reset_index()
df_last = df.groupby('user_id')['ts'].max().reset_index()
#get site_ids of first and last timestamps for each user
df_first = pd.merge(df,df_first, how = 'inner',on = ['user_id','ts'])
df_last = pd.merge(df,df_last,how = 'inner',on = ['user_id','ts'])
# inner join on site ids of first and last gives users whose first and last visit are to the same site
pd.merge(df_last, df_first, how = 'inner' , on = ['user_id','site_id'])['user_id'].nunique()

1670